In [1]:
import pandas as pd
import numpy as np

In [2]:
rewards = pd.read_csv("rewards_table.csv")

In [3]:
rewards

,Unnamed: 0,game,turn,team_id,team_label,unit_id,total_reward,reward_move,reward_fuel_collected,reward_turn_unit,reward_living_city_tiles,reward_death_city_tile,reward_research_point,reward_no_action,reward_fuel_dropped_at_city,reward_win,reward_death_before_end,reward_build_city,reward_transfer,reward_coal_researched
0,0,0,1,0,random,1,0.000000,0.0,0.0,0.0,0.1,-0.0,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,1,1,no_action,2,0.000000,NaN,NaN,0.0,0.1,-0.0,0.05,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,2,0,2,0,random,1,0.000000,0.0,0.0,0.0,0.1,-0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,2,1,no_action,2,0.000000,NaN,NaN,0.0,0.1,-0.0,0.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,4,0,3,0,random,1,0.000000,0.0,0.0,0.0,0.1,-0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113371,113371,299,189,1,no_action,2,0.000000,NaN,NaN,0.0,0.1,-0.0,0.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN
113372,113372,299,190,0,random,1,0.000000,0.0,0.0,0.0,0.1,-0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113373,113373,299,190,1,no_action,2,0.000000,NaN,NaN,0.0,0.1,-0.0,0.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN
113374,113374,299,191,1,no_action,2,20.797368,NaN,NaN,0.0,0.1,-0.0,0.05,0.0,0.0,18.947368,NaN,NaN,NaN,NaN


In [4]:
## cleaning up

# drop columns with unnamed
drop_cols = list(filter(lambda x: "Unnamed" in x, rewards.columns))
rewards.drop(columns=drop_cols, inplace=True)

# fillna
specific_reward_cols = list(filter(lambda x: "reward_" in x, rewards.columns))
for col in specific_reward_cols:
    rewards[col].fillna(0, inplace=True)

rewards = rewards.set_index(["game", "turn", "team_id", "team_label", "unit_id"])

In [5]:
def highlight_positive(v, props=''):
    return props if v <= 0 else None

def highlight_max(s, props=''):
    return np.where((s == np.nanmax(s.values)) & (s.values>0), props, '')

In [9]:
import seaborn as sns
cm_total = sns.cubehelix_palette(start=.5, rot=-.5, as_cmap=True)
cm_total = sns.color_palette("icefire", as_cmap=True)
cm_other = sns.color_palette("vlag", as_cmap=True)

def highlight(df):
    return df.style.background_gradient(vmin=-2, vmax=2, axis=1, subset=specific_reward_cols,cmap=cm_other).\
    background_gradient(axis=0, subset=["total_reward"], vmin=-2, vmax=2, cmap=cm_other)

highlight(rewards.query("game == 9").sort_values(["turn", "team_id"]).iloc[-20:])

# Statistics

In [7]:
# mean cumulative reward of unit per game

rewards.groupby(["game", "team_id", "unit_id"])[["total_reward"]].sum().reset_index().groupby("team_id")[["total_reward"]].mean().rename(columns={"total_reward": "mean_cumulative_reward"}, index={0:"random", 1:"no_action"})

,mean_cumulative_reward
team_id,
random,2.160454
no_action,6.807054


In [8]:

p = rewards.reset_index().query("reward_win > 0").groupby(["game", "turn"])[["team_id"]].first().team_id.mean()
print("Winrate:")
print(f"0: {1-p:%}")
print(f"1: {p:%}")

Winrate:
0: 14.666667%
1: 85.333333%
